In [1]:
import brickschema

g = brickschema.Graph()
g.load_file('chamber_shacl_expanded.ttl')

<Graph identifier=N535ef135ec044051b1d7eea000109c63 (<class 'brickschema.graph.Graph'>)>

In [2]:
# Query test 1
query1 = g.query(
    """SELECT ?ahu ?sat ?damper WHERE {
    ?ahu a brick:Air_Handling_Unit .
    ?ahu brick:hasPoint ?sat .
    ?sat a brick:Supply_Air_Temperature_Sensor .
    ?ahu brick:hasPart ?damper .
    ?damper a brick:Damper
}"""
)
print("\nQuery test 1 results:")
for row in query1:
    print(row)


Query test 1 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#coil_da_temp_10'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#eff_sa_temp_10'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#overhead_sa_temp_10'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_sa_temp_6'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatecha

In [3]:
# Query test 2
query2 = g.query(
    """SELECT * WHERE{ 
    ?sensor rdf:type/rdfs:subClassOf* brick:Water_Temperature_Sensor
}"""
)
print("\nQuery test 2 results:")
for row in query2:
    print(row)


Query test 2 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#T_RWS'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#T_RWR'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#RT_HW'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#return_temp_b_11'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#pchwr_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#schwr_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#supply_temp_b_11'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#pchws_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#schws_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#ST_HW'),)


In [4]:
# Query test 3
query3 = g.query(
    """SELECT ?position WHERE{
    ?ahu a brick:Air_Handling_Unit .
    ?position a brick:Damper_Position_Sensor .
    ?damper a brick:Damper .
    ?zone a brick:HVAC_Zone .
    ?ahu brick:hasPart ?damper .
    ?damper brick:hasPoint ?position .
    ?damper brick:feeds ?zone
    }
    """
)
print("\nQuery test 3 results:")
for row in query3:
    print(row)


Query test 3 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_dmpr_pos_ao_6'),)


In [5]:
# Query test 4
query4 = g.query(
    """SELECT ?sat WHERE{
        ?ahu a brick:Air_Handling_Unit .
        ?sat a brick:Supply_Air_Temperature_Sensor .
        ?ahu brick:hasPoint ?sat
    }
    """
)
print("\nQuery test 4 results:")
for row in query4:
    print(row)


Query test 4 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#sa_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#coil_da_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#eff_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#overhead_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_sa_temp_6'),)


In [6]:
# Query test 5
query5 = g.query(
    """SELECT ?sat WHERE{
        ?ahu a brick:Air_Handling_Unit .
        ?sat a brick:Supply_Air_Temperature_Sensor .
        ?sat brick:isPointOf ?ahu
    }
    """
)
print("\nQuery test 5 results:")
for row in query5:
    print(row)


Query test 5 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#sa_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#coil_da_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#eff_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#overhead_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_sa_temp_6'),)
